##Colab output wrapper

In [30]:
# wrap the output in colab cells
from IPython.display import HTML, display
def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

##Install Transformers

In [3]:
# install transformers with sentencepiece
!pip install transformers[sentencepiece]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.3 MB 32.5 MB/s 
     |████████████████████████████████| 163 kB 56.1 MB/s 
     |████████████████████████████████| 7.6 MB 19.7 MB/s 
     |████████████████████████████████| 1.3 MB 31.3 MB/s 


##Read input file from Google Drive

In [39]:
# open and read the file from google drive
path=r"C:\Users\USER\Desktop\AI_models\Transcript.txt"
path2 ="/content/sample_data/Transcript.txt"
path3 = "/content/sample_data/Essay_Independence.txt"
file = open(path3, "r")
FileContent = file.read().strip()

In [40]:
FileContent

"The day of 15th August 1947 has been embossed in the golden history of India. It is the day when India got its freedom from 200 years of British rule. It was a hard and long struggle in which many freedom fighters and great men sacrificed their lives for our beloved motherland.\n\nIndia celebrates its Independence Day on the 15th of August every year. However, this year, the auspicious day is being celebrated as ‘75 Azadi Ka Amrit Mahotsav 2022’ to commemorate 75 long years of India’s independence. Independence Day, called the Red-Letter Day in the history of our country, is celebrated as a national holiday throughout India.\n\nIndependence Day 2022 helps us remember every one of the penances our political dissidents made to liberate India from British rule. On 15th August 1947, India was announced free from British imperialism and turned into the biggest vote-based system, the largest democracy in the world.\n\nIn this essay on Independence Day, students can track down every one of t

In [41]:
# total characters in the file
len(FileContent)

8816

##Load the Model and Tokenizer

In [42]:
# import and initialize the tokenizer and model from the checkpoint
#from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
# checkpoint = "sshleifer/distilbart-cnn-12-6"
# tokenizer = AutoTokenizer.from_pretrained(checkpoint)
# model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

In [43]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("facebook/bart-large-cnn")

model = AutoModelForSeq2SeqLM.from_pretrained("facebook/bart-large-cnn")

##Some model statistics

In [44]:
# max tokens including the special tokens
tokenizer.model_max_length

1024

In [45]:
# max tokens excluding the special tokens
tokenizer.max_len_single_sentence

1022

In [46]:
# number of special tokens
tokenizer.num_special_tokens_to_add()

2

##Convert file content to sentences

In [47]:
# extract the sentences from the document
import nltk
nltk.download('punkt')
sentences = nltk.tokenize.sent_tokenize(FileContent)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [48]:
sentences[3]

'India celebrates its Independence Day on the 15th of August every year.'

In [17]:
# find the  tokens in the sentence
#[len(tokenizer.tokenize(sentence)) for sentence in sentences]


In [49]:
# find the max tokens in the longest sentence
max([len(tokenizer.tokenize(sentence)) for sentence in sentences])


64

##Create the chunks

In [51]:
# initialize
length = 0
chunk = ""
chunks = []
count = -1
for sentence in sentences:
  count += 1
  combined_length = len(tokenizer.tokenize(sentence)) + length # add the no. of sentence tokens to the length counter
  if combined_length  <= tokenizer.max_len_single_sentence: # if it doesn't exceed
    print("Part 1. In IF condition")
    print("IF: ",sentence)
    print("IF: ",len(tokenizer.tokenize(sentence)))
    chunk += sentence + " " # add the sentence to the chunk
    length = combined_length # update the length counter
    # if it is the last sentence
    if count == len(sentences) - 1:
      print("&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&&IT IS THE LAST SENTENCE")
      chunks.append(chunk.strip()) # save the chunk
  else: 
    print("################part 2 in else condition###############")
    print("ELSE: ",sentence)
    print("ELSE: ",len(tokenizer.tokenize(sentence)))
    chunks.append(chunk.strip()) # save the chunk   
    # reset 
    length = 0 
    chunk = ""
    # take care of the overflow sentence
    chunk += sentence + " "
    length = len(tokenizer.tokenize(sentence))
len(chunks)

Part 1. In IF condition
IF:  The day of 15th August 1947 has been embossed in the golden history of India.
IF:  19
Part 1. In IF condition
IF:  It is the day when India got its freedom from 200 years of British rule.
IF:  16
Part 1. In IF condition
IF:  It was a hard and long struggle in which many freedom fighters and great men sacrificed their lives for our beloved motherland.
IF:  24
Part 1. In IF condition
IF:  India celebrates its Independence Day on the 15th of August every year.
IF:  14
Part 1. In IF condition
IF:  However, this year, the auspicious day is being celebrated as ‘75 Azadi Ka Amrit Mahotsav 2022’ to commemorate 75 long years of India’s independence.
IF:  39
Part 1. In IF condition
IF:  Independence Day, called the Red-Letter Day in the history of our country, is celebrated as a national holiday throughout India.
IF:  26
Part 1. In IF condition
IF:  Independence Day 2022 helps us remember every one of the penances our political dissidents made to liberate India from 

2

##Some checks

In [52]:
[len(tokenizer.tokenize(c)) for c in chunks]


[996, 771]

In [53]:
[len(tokenizer(c).input_ids) for c in chunks]

[998, 773]

##With special tokens added

In [54]:
sum([len(tokenizer(c).input_ids) for c in chunks])

1771

In [55]:
len(tokenizer(FileContent).input_ids)

Token indices sequence length is longer than the specified maximum sequence length for this model (1820 > 1024). Running this sequence through the model will result in indexing errors


1820

##Without special tokens added

In [56]:
sum([len(tokenizer.tokenize(c)) for c in chunks])

1767

In [57]:
len(tokenizer.tokenize(FileContent))

1818

##Get the inputs

In [58]:
# inputs to the model
inputs = [tokenizer(chunk, return_tensors="pt") for chunk in chunks]


In [59]:
import warnings
warnings.filterwarnings("ignore")

##Output

In [60]:
for input in inputs:
  output = model.generate(**input)
  print(tokenizer.decode(*output,max_new_tokens=1024, skip_special_tokens=True))

India celebrates its Independence Day on the 15th of August every year. This year, the auspicious day is being celebrated as ‘75 Azadi Ka Amrit Mahotsav 2022’ to commemorate 75 long years of India’s independence. In this essay on Independence Day, students can track down every one of the significant subtleties of India's Independence history.
Independence Day is celebrated with great joy across the country. People hold meetings, fly the tricolour flag, and sing the National Anthem. Tributes are paid to the freedom fighters who sacrificed their lives during the struggle. The young minds are infused with a sense of patriotism and nationalistic feelings. The country is celebrating 75 Azadi Ka Amrit Mahotsav.
